In [17]:
from   symcircuit.symbolic_solver.domains    import Circuit, ExperimentResult
from   symcircuit.symbolic_solver.solver     import Circuit_Solver, Impedance_Analyzer
from   symcircuit.demo.multiple_feedback     import Current_Mode_Multiple_Feedback as CMMF
import sympy

# Manual Runs

In [18]:
circuit = CMMF.circuit
circuit.hasSolution()

True

In [19]:
circuit.impedances, circuit.impedancesToDisconnect

([Z_1, Z_2, Z_3, Z_4, Z_5], [Z_1, Z_2, Z_3, Z_4, Z_5])

In [20]:
for i, eq in enumerate(circuit.nodal_equations, 1):
    print(f"{i} - {eq}")

1 - Eq(I2, (V2 - Va)/Z_5 + (-V1 + V2)/Z_2)
2 - Eq(I1, -V1/Z_4 + (-V1 + Va)/Z_3 + (-V1 + V2)/Z_2)
3 - Eq(I1, V1/Z_1)
4 - Eq(V2, 0)


In [21]:
circuit.solve_for

[V1, V2, Va, I1, I2]

## Circuit Solver

In [22]:
solver = Circuit_Solver(circuit=circuit, _output= sympy.symbols("I1 0"), _input=sympy.symbols("I2 0"), transmissionMatrixType="NA")

In [23]:
solver.impedances

[Z_1, Z_2, Z_3, Z_4, Z_5]

In [24]:
solver.impedancesToDisconnect

[Z_1, Z_2, Z_3, Z_4, Z_5]

In [25]:
solver.solve()

====Solving the Circuit====
--- Computing the possible impedance connections for [Z_1, Z_2, Z_3, Z_4, Z_5] ---
--- Impedance connections stored in Circuit_Solver.impedanceConnections---
 ----------------------------
1 - Solving for (I1 - 0) / (I2 - 0)
2 - Intermediate Variables: [V1, V2, Va, I1, I2]
 ----------------------------
3 - solved the base transfer function (symbolic [T])
-- Single-ended input/output
processing combo size 1
processing combo size 2
processing combo size 3
processing combo size 4
processing combo size 5
=====*Circuit Solved*=====


In [26]:
solver.solutions

[{I1: -I2*Z_2*Z_4*Z_5/(Z_1*Z_2*Z_3 + Z_1*Z_2*Z_4 + Z_1*Z_3*Z_4 + Z_1*Z_4*Z_5 + Z_2*Z_3*Z_4),
  V1: -I2*Z_1*Z_2*Z_4*Z_5/(Z_1*Z_2*Z_3 + Z_1*Z_2*Z_4 + Z_1*Z_3*Z_4 + Z_1*Z_4*Z_5 + Z_2*Z_3*Z_4),
  V2: 0,
  Va: I2*(-Z_1*Z_2*Z_3*Z_5 - Z_1*Z_2*Z_4*Z_5 - Z_1*Z_3*Z_4*Z_5 - Z_2*Z_3*Z_4*Z_5)/(Z_1*Z_2*Z_3 + Z_1*Z_2*Z_4 + Z_1*Z_3*Z_4 + Z_1*Z_4*Z_5 + Z_2*Z_3*Z_4)}]

In [27]:
solver.solutions[0][sympy.symbols("Va")]


I2*(-Z_1*Z_2*Z_3*Z_5 - Z_1*Z_2*Z_4*Z_5 - Z_1*Z_3*Z_4*Z_5 - Z_2*Z_3*Z_4*Z_5)/(Z_1*Z_2*Z_3 + Z_1*Z_2*Z_4 + Z_1*Z_3*Z_4 + Z_1*Z_4*Z_5 + Z_2*Z_3*Z_4)

In [28]:
symcircuit_sol = solver.baseHs
symcircuit_sol

-Z_2*Z_4*Z_5/(Z_1*Z_2*Z_3 + Z_1*Z_2*Z_4 + Z_1*Z_3*Z_4 + Z_1*Z_4*Z_5 + Z_2*Z_3*Z_4)

In [29]:
Y1, Y2, Y3, Y4, Y5 = sympy.symbols("Y1, Y2, Y3, Y4, Y5")
Z1, Z2, Z3, Z4, Z5 = sympy.symbols("Z1, Z2, Z3, Z4, Z5")

tf_from_the_original_paper = -(Y1*Y3)/((Y1+Y2+Y3+Y4)*Y5 + Y2*Y3)
tf_from_the_original_paper

-Y1*Y3/(Y2*Y3 + Y5*(Y1 + Y2 + Y3 + Y4))

In [30]:
new_form = tf_from_the_original_paper.subs({
    Y1 : 1/Z1,
    Y2 : 1/Z2,
    Y3 : 1/Z3,
    Y4 : 1/Z4,
    Y5 : 1/Z5,
})
new_form = new_form.simplify()
new_form

-Z2*Z4*Z5/(Z1*Z2*Z3 + Z1*Z2*Z4 + Z1*Z3*Z4 + Z1*Z4*Z5 + Z2*Z3*Z4)

## Impedance Analyzer

In [31]:
analysis = Impedance_Analyzer("TEST_CMMF", circuit_solver=solver)
print(analysis.isCircuitSolved())
analysis

True


In [32]:
for block in analysis.impedance_blocks:
    print(f"{block.name} - {block.symbol} - allowed connections {block.allowedConnections}")

1 - Z_1 - allowed connections [R_1, 1/(C_1*s), R_1 + 1/(C_1*s), R_1/(C_1*R_1*s + 1)]
2 - Z_2 - allowed connections [R_2, 1/(C_2*s), R_2 + 1/(C_2*s), R_2/(C_2*R_2*s + 1)]
3 - Z_3 - allowed connections [R_3, 1/(C_3*s), R_3 + 1/(C_3*s), R_3/(C_3*R_3*s + 1)]
4 - Z_4 - allowed connections [R_4, 1/(C_4*s), R_4 + 1/(C_4*s), R_4/(C_4*R_4*s + 1)]
5 - Z_5 - allowed connections [R_5, 1/(C_5*s), R_5 + 1/(C_5*s), R_5/(C_5*R_5*s + 1)]


In [33]:
analysis.computeTFs(comboKey="Z1_Z2_Z3_Z4_Z5")

combo key = Z1_Z2_Z3_Z4_Z5
Zcombos


Getting the TFs (CG): 100%|██████████| 1024/1024 [00:10<00:00, 99.72combo/s]

Number of transfer functions found: 1024


In [34]:
analysis.classifier.classifyBiQuadFilters()

Computing Biquad filter parameters: 100%|██████████| 1024/1024 [00:19<00:00, 52.65filter/s]


In [35]:
analysis.classifier.classifications

[FilterClassification(zCombo=(R_1, R_2, R_3, R_4, R_5), transferFunc=-R_2*R_4*R_5/(R_1*R_2*R_3 + R_1*R_2*R_4 + R_1*R_3*R_4 + R_1*R_4*R_5 + R_2*R_3*R_4), valid=False, fType=INVALID-ORDER, parameters=None),
 FilterClassification(zCombo=(R_1, R_2, R_3, R_4, 1/(C_5*s)), transferFunc=-R_2*R_4/(R_1*R_4 + s*(C_5*R_1*R_2*R_3 + C_5*R_1*R_2*R_4 + C_5*R_1*R_3*R_4 + C_5*R_2*R_3*R_4)), valid=False, fType=INVALID-ORDER, parameters=None),
 FilterClassification(zCombo=(R_1, R_2, R_3, R_4, R_5 + 1/(C_5*s)), transferFunc=(-C_5*R_2*R_4*R_5*s - R_2*R_4)/(R_1*R_4 + s*(C_5*R_1*R_2*R_3 + C_5*R_1*R_2*R_4 + C_5*R_1*R_3*R_4 + C_5*R_1*R_4*R_5 + C_5*R_2*R_3*R_4)), valid=False, fType=INVALID-ORDER, parameters=None),
 FilterClassification(zCombo=(R_1, R_2, R_3, R_4, R_5/(C_5*R_5*s + 1)), transferFunc=-R_2*R_4*R_5/(R_1*R_2*R_3 + R_1*R_2*R_4 + R_1*R_3*R_4 + R_1*R_4*R_5 + R_2*R_3*R_4 + s*(C_5*R_1*R_2*R_3*R_5 + C_5*R_1*R_2*R_4*R_5 + C_5*R_1*R_3*R_4*R_5 + C_5*R_2*R_3*R_4*R_5)), valid=False, fType=INVALID-ORDER, paramete

In [36]:
classifications = analysis.classifier.summarizeFilterType()

HP : 25
BP : 70
LP : 25
BS : 0
GE : 0
AP : 0
INVALID-NUMER : 192
INVALID-WZ : 86
INVALID-ORDER : 626
PolynomialError : 0


In [37]:
analysis.reportSummary("CMMF_BiQuad", "Z1_Z2_Z3_Z4_Z5")
analysis.compilePDF()

LaTeX report generated and saved to: Runs/TEST_CMMF/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_summary.tex

=== Compiling the reports to PDF ===
----** compiling Runs/TEST_CMMF/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_summary.tex **----
PDF generated: Runs/TEST_CMMF/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_summary.pdf
=== Compiling DONE ===



In [38]:
analysis.classifier.classifyFilter(filterOrder="FirstOrder")

Computing filter parameters for FirstOrder: 100%|██████████| 1024/1024 [00:05<00:00, 175.35filter/s]


In [39]:
classifications = analysis.classifier.summarizeFilterType()

HP : 25
BP : 38
LP : 25
BS : 0
GE : 0
AP : 0
INVALID-NUMER : 0
INVALID-WZ : 0
INVALID-ORDER : 936
PolynomialError : 0


In [40]:
analysis.reportSummary("CMMF_FirstOrder", "Z1_Z2_Z3_Z4_Z5")
analysis.compilePDF()

LaTeX report generated and saved to: Runs/TEST_CMMF/CMMF_FirstOrder_Z1_Z2_Z3_Z4_Z5/CMMF_FirstOrder_Z1_Z2_Z3_Z4_Z5_summary.tex

=== Compiling the reports to PDF ===
----** compiling Runs/TEST_CMMF/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_summary.tex **----
PDF generated: Runs/TEST_CMMF/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5/CMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_summary.pdf
----** compiling Runs/TEST_CMMF/CMMF_FirstOrder_Z1_Z2_Z3_Z4_Z5/CMMF_FirstOrder_Z1_Z2_Z3_Z4_Z5_summary.tex **----
PDF generated: Runs/TEST_CMMF/CMMF_FirstOrder_Z1_Z2_Z3_Z4_Z5/CMMF_FirstOrder_Z1_Z2_Z3_Z4_Z5_summary.pdf
=== Compiling DONE ===



# Automated Search

In [41]:
from   symcircuit.symbolic_solver.solver     import run_experiment

circuit = CMMF.circuit


experiment_result = run_experiment("TEST_CMMF_Automated",
               T_type="NA",
               circuit=circuit,
               minNumOfActiveImpedances=3,
               maxNumOfActiveImpedances=5,
               impedanceKeysOverwrite= None,
               outputFrom= ["I1", "0"],
               inputFrom=["I2", "0"],
               loadHistory=True
               )



!!! could not ressolve the path to the pkl file (found 0)

Solving the circuit for the first time
====Solving the Circuit====
--- Computing the possible impedance connections for [Z_1, Z_2, Z_3, Z_4, Z_5] ---
--- Impedance connections stored in Circuit_Solver.impedanceConnections---
 ----------------------------
1 - Solving for (I1 - 0) / (I2 - 0)
2 - Intermediate Variables: [V1, V2, Va, I1, I2]
 ----------------------------
3 - solved the base transfer function (symbolic [T])
-- Single-ended input/output
processing combo size 1
processing combo size 2
processing combo size 3
processing combo size 4
processing combo size 5
=====*Circuit Solved*=====

=== ExperimentResult saved successfully to Runs/TEST_CMMF_Automated_NA/results_circuit_solution.pkl - 4.947 kb ===

** Loading the circuit history
could not ressolve the path to the pkl file (found 0)
** Found 0 keys already computed
Performing a systematic search: min_num_of_active_z = 3 max_num_of_active_z = 5
Experiment will be ran for

Getting the TFs (CG): 100%|██████████| 64/64 [00:00<00:00, 1258.92combo/s]


Number of transfer functions found: 64


Computing Biquad filter parameters: 100%|██████████| 64/64 [00:00<00:00, 2663.00filter/s]

HP : 0
BP : 4
LP : 0
BS : 0
GE : 0
AP : 0
INVALID-NUMER : 0
INVALID-WZ : 0
INVALID-ORDER : 60
PolynomialError : 0
LaTeX report generated and saved to: Runs/TEST_CMMF_Automated_NA/TEST_CMMF_Automated_NA_Z1_Z2_Z3/TEST_CMMF_Automated_NA_Z1_Z2_Z3_summary.tex

=== Compiling the reports to PDF ===
----** compiling Runs/TEST_CMMF_Automated_NA/TEST_CMMF_Automated_NA_Z1_Z2_Z3/TEST_CMMF_Automated_NA_Z1_Z2_Z3_summary.tex **----


...

HP : 0
BP : 0
LP : 0
BS : 0
GE : 0
AP : 0
INVALID-NUMER : 0
INVALID-WZ : 0
INVALID-ORDER : 256
PolynomialError : 0
!!! INVALID Z_arr !!!

=== Compiling the reports to PDF ===
=== Compiling DONE ===


=== ExperimentResult saved successfully to Runs/TEST_CMMF_Automated_NA/results.pkl - 354.48 kb ===

==> Running the TEST_CMMF_Automated_NA Experiment for Z1_Z2_Z3_Z4_Z5 (progress: 1/1 combo size: 3/3)

combo key = Z1_Z2_Z3_Z4_Z5
Zcombos


Getting the TFs (CG): 100%|██████████| 1024/1024 [00:10<00:00, 94.51combo/s]


Number of transfer functions found: 1024


Computing Biquad filter parameters: 100%|██████████| 1024/1024 [00:19<00:00, 53.12filter/s]


HP : 25
BP : 70
LP : 25
BS : 0
GE : 0
AP : 0
INVALID-NUMER : 192
INVALID-WZ : 86
INVALID-ORDER : 626
PolynomialError : 0
LaTeX report generated and saved to: Runs/TEST_CMMF_Automated_NA/TEST_CMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5/TEST_CMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5_summary.tex

=== Compiling the reports to PDF ===
----** compiling Runs/TEST_CMMF_Automated_NA/TEST_CMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5/TEST_CMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5_summary.tex **----
PDF generated: Runs/TEST_CMMF_Automated_NA/TEST_CMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5/TEST_CMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5_summary.pdf
=== Compiling DONE ===


=== ExperimentResult saved successfully to Runs/TEST_CMMF_Automated_NA/results.pkl - 996.518 kb ===

<----> END OF EXPERIMENT <---->
Impedance Keys analyzed (count: 16): 
[{'Z1_Z2_Z3': (Z_1, Z_2, Z_3),
  'Z1_Z2_Z4': (Z_1, Z_2, Z_4),
  'Z1_Z2_Z5': (Z_1, Z_2, Z_5),
  'Z1_Z3_Z4': (Z_1, Z_3, Z_4),
  'Z1_Z3_Z5': (Z_1, Z_3, Z_5),
  'Z1_Z4_Z5': (Z_1, Z_4, Z_5),
  'Z2_Z3_Z4': (Z_2, Z_3, Z_4),
  'Z

In [42]:
classifications, tf = experiment_result.get("Z1_Z2_Z3_Z4_Z5")
tf

-Z_2*Z_4*Z_5/(Z_1*Z_2*Z_3 + Z_1*Z_2*Z_4 + Z_1*Z_3*Z_4 + Z_1*Z_4*Z_5 + Z_2*Z_3*Z_4)

In [43]:
type(classifications), type(classifications[-1])

(list, symcircuit.symbolic_solver.domains.Filter_Classification)

In [44]:
df_classifications = experiment_result.flatten_classifications()
df_classifications

,impedance_key,zCombo,transferFunc,transferFunc_latex,valid,fType,parameters,filterOrder
0,Z1_Z2_Z3,"(R_1, R_2, R_3, oo, oo)",-R_2/R_1,- \frac{R_{2}}{R_{1}},False,INVALID-ORDER,None,BiQuad
1,Z1_Z2_Z3,"(R_1, R_2, 1/(C_3*s), oo, oo)",-R_2/R_1,- \frac{R_{2}}{R_{1}},False,INVALID-ORDER,None,BiQuad
2,Z1_Z2_Z3,"(R_1, R_2, R_3 + 1/(C_3*s), oo, oo)",-R_2/R_1,- \frac{R_{2}}{R_{1}},False,INVALID-ORDER,None,BiQuad
3,Z1_Z2_Z3,"(R_1, R_2, R_3/(C_3*R_3*s + 1), oo, oo)",-R_2/R_1,- \frac{R_{2}}{R_{1}},False,INVALID-ORDER,None,BiQuad
4,Z1_Z2_Z3,"(R_1, 1/(C_2*s), R_3, oo, oo)",-1/(C_2*R_1*s),- \frac{1}{C_{2} R_{1} s},False,INVALID-ORDER,None,BiQuad
...,...,...,...,...,...,...,...,...
2939,Z1_Z2_Z3_Z4_Z5,"(R_1/(C_1*R_1*s + 1), R_2/(C_2*R_2*s + 1), R_3...",(-C_1*C_3*C_4*R_1*R_2*R_3*R_4*R_5*s**3 - R_2*R...,\frac{- C_{1} C_{3} C_{4} R_{1} R_{2} R_{3} R_...,False,INVALID-ORDER,None,BiQuad
2940,Z1_Z2_Z3_Z4_Z5,"(R_1/(C_1*R_1*s + 1), R_2/(C_2*R_2*s + 1), R_3...",(-C_1*C_3*R_1*R_2*R_3*R_4*R_5*s**2 - R_2*R_4*R...,\frac{- C_{1} C_{3} R_{1} R_{2} R_{3} R_{4} R_...,False,INVALID-WZ,{'Q': C_2*C_3*R_2*R_3*R_5*sqrt(1/(C_2*C_3*R_4*...,BiQuad
2941,Z1_Z2_Z3_Z4_Z5,"(R_1/(C_1*R_1*s + 1), R_2/(C_2*R_2*s + 1), R_3...",(-C_1*C_3*R_1*R_2*R_3*R_4*s**2 - R_2*R_4 + s*(...,\frac{- C_{1} C_{3} R_{1} R_{2} R_{3} R_{4} s^...,False,INVALID-WZ,{'Q': (C_1*C_5*R_1*R_2*R_3*R_4*sqrt(1/(C_1*C_5...,BiQuad
2942,Z1_Z2_Z3_Z4_Z5,"(R_1/(C_1*R_1*s + 1), R_2/(C_2*R_2*s + 1), R_3...",(-C_1*C_3*C_5*R_1*R_2*R_3*R_4*R_5*s**3 - R_2*R...,\frac{- C_{1} C_{3} C_{5} R_{1} R_{2} R_{3} R_...,False,INVALID-ORDER,None,BiQuad


In [45]:
df_tfs = experiment_result.flatten_tfs()
df_tfs

,impedance_key,transferFunc,transferFunc_latex
0,Z1_Z2_Z3,-Z_2/Z_1,- \frac{Z_{2}}{Z_{1}}
1,Z1_Z2_Z4,0,0
2,Z1_Z2_Z5,0,0
3,Z1_Z3_Z4,-oo*sign(Z_4/(Z_1*Z_3 + Z_1*Z_4 + Z_3*Z_4)),- \infty \operatorname{sign}{\left(\frac{Z_{4}...
4,Z1_Z3_Z5,-Z_5/(Z_1 + Z_3),- \frac{Z_{5}}{Z_{1} + Z_{3}}
5,Z1_Z4_Z5,0,0
6,Z2_Z3_Z4,0,0
7,Z2_Z3_Z5,0,0
8,Z2_Z4_Z5,0,0
9,Z3_Z4_Z5,0,0


In [46]:
experiment_result.to_csv()

Writting TEST_CMMF_Automated_NA to csv files
flattened all the classifications to Runs/TEST_CMMF_Automated_NA/classifications_.csv
flattened all the classifications to Runs/TEST_CMMF_Automated_NA/tfs_.csv
